In [1]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  q


True

In [10]:
from nltk.corpus import brown
print(brown.words())
wrds=[]
freq={}
punct="!\"#$%&'()*+,-—./:;<=>?@[\]^_`{|}~"
for w in brown.words():
    curr=w
    for p in punct:
        curr=curr.replace(p,'')
    if curr != "":
        wrds.append(w.lower())
        if curr in freq:
            freq[curr] += 1
        else:
            freq[curr] =1
# print(" ".join(wrds))
# V, C according to frequency
freqlist=[]
for key in freq:
    freqlist.append([key,freq[key]])
freqlist=sorted(freqlist,key=lambda x:x[1])
# print(freqlist)
W=[] # target words
Wmap={} # <5000--W;  <1000--C
for i in range(1,5001):
    W.append(freqlist[-i][0])
    Wmap[freqlist[-i][0]]=i-1
C=[] # surrounding
for i in range(1,1001):
    C.append(freqlist[-i][0])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]


In [37]:
# nwc
import numpy as np
wrd_idx=[]
for i in range(len(wrds)):
    if wrds[i] in Wmap:
        wrd_idx.append(Wmap[wrds[i]])
    else:
        wrd_idx.append(-1)

nwc=[[0 for j in range(len(C))]for i in range(len(W))]
c_count=[0 for j in range(len(C))]
n=len(wrds)
print(np.array(nwc).shape)
for i in range(n):
    wordindex=wrd_idx[i]
    if wordindex !=-1:
        for j in range(-2,3):
            if j!=0 and 0 <= i+j < n:
                cindex=wrd_idx[i+j]
                if cindex!=-1 and cindex<1000:
                    nwc[wordindex][cindex] += 1
                    c_count[cindex] += 1

(5000, 1000)


In [64]:
import copy
c_count=np.array(c_count)
nwc=np.array(nwc)

P_c=c_count/sum(c_count)
sum_nwc=sum(nwc.T)
# P_c_w=nwc/sum_nwc
P_w_c=[0 for i in range(len(nwc))]
for i in range(len(sum_nwc)):
    if sum_nwc[i] != 0:
        P_w_c[i] = nwc[i] / float(sum_nwc[i])
    else:
        if max(nwc[i])>0:
            print("break! error !")
            print(max(nwc[i]))
            break
        P_w_c[i] = np.array([0 for j in range(len(C))])
print(len(P_w_c))
print(len(P_w_c[0]))
print(len(P_c))

5000
1000
1000


In [122]:
# phi_c_w
phi_w_c=[[0.0 for j in range(len(C))]for i in range(len(W))]
for i in range(len(W)):
    for j in range(len(C)):
        if P_c[j]>0 and P_w_c[i][j]>0:
            frac=P_w_c[i][j]/P_c[j]
            if res>1:
                log=np.log(frac)
                phi_w_c[i][j]=log
            else:
                phi_w_c[i][j]=0.0
        else:
            phi_w_c[i][j]=0.0
phi_w_c=np.array(phi_w_c)
print(phi_w_c.shape)

(5000, 1000)


In [144]:
# PCA
# covariance matrix
covmatr=np.cov(phi_w_c.T)
print("covmatr",covmatr.shape)

# eigenvalue
val, vect = LA.eig(covmatr)
print("vect",vect.shape)
# sort to get the largest
zipped=list(zip(val,vect.T)) # already tranversed
zipped=sorted(zipped,key=lambda x:-x[0])

# mean
mean=sum(phi_w_c)/len(phi_w_c)
print("mean",mean.shape)

# PCA
embed_n=100
ut=[]
for i in range(embed_n):
    ut.append(zipped[i][1])
ut=np.array(ut)

embed_x=[]
for i in range(len(phi_w_c)):
    x=ut @ (phi_w_c[i]-mean)
    embed_x.append(x)
embed_x=np.array(embed_x)
print("embed_x",embed_x.shape)

covmatr (1000, 1000)
vect (1000, 1000)
mean (1000,)
embed_x (5000, 100)


In [179]:
# nearest neighbor
def dis_cos(a,b):
    res=1 - np.dot(a,b)/math.sqrt(np.dot(a,a)*np.dot(b,b))
    return res

def nn(index,data,disfuc):
    idx=0
    x=data[index]
    if index == 0:
        dis=disfuc(x,data[1])
    else:
        dis=disfuc(x,data[0])
    # print(len(data))
    for i in range(len(data)):
        if i == index:
            continue
        dis_i=disfuc(x,data[i])
        if dis > dis_i:
            # print(i,dis)
            dis=dis_i
            idx=i
    return idx,dis

import random
for i in range(25):
    index=random.randrange(5000)
    idx,dis=nn(index,embed_x,dis_cos)
    print(W[index],"==>",W[idx])

taxpayers ==> statistics
killing ==> lonely
See ==> The
Little ==> The
First ==> The
sin ==> images
sequence ==> connection
average ==> total
slowly ==> suddenly
mention ==> happen
1958 ==> 1960
external ==> educational
liquor ==> operator
lovely ==> beautiful
lightly ==> shoulder
continuously ==> 20
stayed ==> sat
transfer ==> interpretation
Five ==> The
formula ==> binomial
familiar ==> convinced
lives ==> family
government ==> state
uneasy ==> ugly
chance ==> reason


In [184]:
# clustering
from sklearn.cluster import KMeans
clst_n=100
kmeans = KMeans(n_clusters=clst_n,random_state=0).fit(embed_x)
labelnum=kmeans.labels_
clst=[[] for i in range(clst_n)]
for i in range(len(labelnum)):
    clst[labelnum[i]].append(i)

In [188]:
for i in range(len(clst)):
    print("clst",i,":")
    for j in clst[i]:
        print(W[j],end=' , ')
    print()

clst 0 :
through , between , under , against , without , upon , within , 
clst 1 :
The , as , I , which , He , one , all , It , In , time , A , But , This , Af , And , She , There , They , Mr , If , We , For , When , New , American , As , Mrs , His , You , What , States , United , To , One , No , At , didnt , dont , John , That , Then , On , These , Of , York , God , With , All , Now , Im , State , President , After , Its , Miss , So , C , How , Some , Washington , By , An , English , America , Ill , Dr , House , Not , General , West , Even , South , Well , B , Since , couldnt , However , Thus , US , St , Department , From , National , Why , War , My , England , Congress , Secretary , cant , Her , wasnt , William , Government , Christian , North , Here , East , Kennedy , French , S , Yet , Island , Federal , City , During , George , Soviet , J , Ive , Church , wouldnt , University , Although , E , Just , While , D , First , British , Europe , Each , Aj , mans , Act , Brown , Such , Whi